# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [36]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [37]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [38]:
y_train.shape

(1051, 1)

In [4]:
y_val.shape

(263, 1)

In [39]:
X_test_cat.shape

(146, 260)

In [40]:
X_test_numeric.shape

(146, 36)

In [6]:
y_test.shape

(146, 1)

In [ ]:
y_test

In [7]:
y_test.shape

(146, 1)

In [41]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)


# Number of features
n_features = X_train.shape[1]
n_features

296

In [42]:
X_test.shape

(146, 296)

As a refresher, preview the training data: 

In [9]:
y_train.head()

,SalePrice
0,180000
1,230000
2,230000
3,236500
4,310000


In [10]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [43]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units

baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
# Hidden layer with 50 units

baseline_model.add(layers.Dense(50, activation='relu'))


# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 106us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
1051/1051 [==============================] - 0s 59us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
1051/1051 [==============================] - 0s 66us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
1051/1051 [==============================] - 0s 63us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
1051/1051 [==============================] - 0s 58us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
1051/1051 [==============================] - 0s 56us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
1051/1051 [==============================] - 0s 58us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
1051/1051 [==============================] - 0s 55us/step - loss: nan - 

1051/1051 [==============================] - 0s 88us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 67/150
1051/1051 [==============================] - 0s 75us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 68/150
1051/1051 [==============================] - 0s 71us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
1051/1051 [==============================] - 0s 63us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
1051/1051 [==============================] - 0s 80us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
1051/1051 [==============================] - 0s 79us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
1051/1051 [==============================] - 0s 70us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
1051/1051 [==============================] - 0s 57us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150


1051/1051 [==============================] - 0s 80us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 132/150
1051/1051 [==============================] - 0s 76us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 133/150
1051/1051 [==============================] - 0s 75us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 134/150
1051/1051 [==============================] - 0s 62us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 135/150
1051/1051 [==============================] - 0s 67us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 136/150
1051/1051 [==============================] - 0s 79us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
1051/1051 [==============================] - 0s 71us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
1051/1051 [==============================] - 0s 78us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [56]:
# Numeric column names
numeric_columns = X_train_numeric.columns

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

In [47]:
X_train.shape

(1051, 296)

In [55]:
X_test_scaled.shape

(146, 36)

In [52]:
X_test.shape

(263, 296)

In [48]:
y_test.shape

(146, 1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [57]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [58]:
# Train the model 
normalized_input_model.fit(X_train, y_train,
                          batch_size=32,
                          epochs=150, 
                          validation_data=(X_val, y_val))


Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 153us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
1051/1051 [==============================] - 0s 65us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
1051/1051 [==============================] - 0s 61us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
1051/1051 [==============================] - 0s 53us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
1051/1051 [==============================] - 0s 56us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
1051/1051 [==============================] - 0s 55us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
1051/1051 [==============================] - 0s 55us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
1051/1051 [==============================] - 0s 60us/step - loss: nan - 

1051/1051 [==============================] - 0s 56us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 67/150
1051/1051 [==============================] - 0s 68us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 68/150
1051/1051 [==============================] - 0s 78us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
1051/1051 [==============================] - 0s 89us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
1051/1051 [==============================] - 0s 83us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
1051/1051 [==============================] - 0s 64us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
1051/1051 [==============================] - 0s 72us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
1051/1051 [==============================] - 0s 64us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150


1051/1051 [==============================] - 0s 58us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 132/150
1051/1051 [==============================] - 0s 57us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 133/150
1051/1051 [==============================] - 0s 61us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 134/150
1051/1051 [==============================] - 0s 79us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 135/150
1051/1051 [==============================] - ETA: 0s - loss: nan - mse: na - 0s 84us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 136/150
1051/1051 [==============================] - 0s 80us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
1051/1051 [==============================] - 0s 72us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
1051/1051 [==============================] - 0s 65us/step - loss: nan - mse: nan - val

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [15]:
y_train.shape

(1051, 1)

In [68]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [69]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
# normalized_model.fit(X_train, 
#                      y_train_scaled,
#                      batch_size=32,
#                      epochs=150, 
#                      validation_data=(X_val, y_val_scaled))
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))


Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 125us/step - loss: 0.4495 - mse: 0.4495 - val_loss: 0.1712 - val_mse: 0.1712
Epoch 2/150
1051/1051 [==============================] - 0s 64us/step - loss: 0.2342 - mse: 0.2342 - val_loss: 0.1550 - val_mse: 0.1550
Epoch 3/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.1943 - mse: 0.1943 - val_loss: 0.1825 - val_mse: 0.1825
Epoch 4/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.1753 - mse: 0.1753 - val_loss: 0.1358 - val_mse: 0.1358
Epoch 5/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.1569 - mse: 0.1569 - val_loss: 0.1343 - val_mse: 0.1343
Epoch 6/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.1380 - mse: 0.1380 - val_loss: 0.1298 - val_mse: 0.1298
Epoch 7/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.1337 - mse: 0.1337 - val_loss: 0.1313 - val_mse: 0.1313


1051/1051 [==============================] - 0s 58us/step - loss: 0.0234 - mse: 0.0234 - val_loss: 0.1367 - val_mse: 0.1367
Epoch 61/150
1051/1051 [==============================] - 0s 66us/step - loss: 0.0232 - mse: 0.0232 - val_loss: 0.1353 - val_mse: 0.1353
Epoch 62/150
1051/1051 [==============================] - 0s 62us/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.1369 - val_mse: 0.1369
Epoch 63/150
1051/1051 [==============================] - 0s 68us/step - loss: 0.0226 - mse: 0.0226 - val_loss: 0.1372 - val_mse: 0.1372
Epoch 64/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0223 - mse: 0.0223 - val_loss: 0.1366 - val_mse: 0.1366
Epoch 65/150
1051/1051 [==============================] - 0s 66us/step - loss: 0.0219 - mse: 0.0219 - val_loss: 0.1369 - val_mse: 0.1369
Epoch 66/150
1051/1051 [==============================] - 0s 69us/step - loss: 0.0211 - mse: 0.0211 - val_loss: 0.1371 - val_mse: 0.1371
Epoch 67/150
1051/1051 [==============================

Epoch 120/150
1051/1051 [==============================] - 0s 62us/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1379 - val_mse: 0.1379
Epoch 121/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1367 - val_mse: 0.1367
Epoch 122/150
1051/1051 [==============================] - 0s 89us/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1367 - val_mse: 0.1367
Epoch 123/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1367 - val_mse: 0.1367
Epoch 124/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1353 - val_mse: 0.1353
Epoch 125/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.1365 - val_mse: 0.1365
Epoch 126/150
1051/1051 [==============================] - 0s 68us/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1358 - val_mse: 0.1358
Epoch 127/150
1051/1051 [=========

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [70]:
# Evaluate the model on training data
#evaluate model
results_train = normalized_model.evaluate(X_train, y_train_scaled)
print('----------')
print(f'Training Loss: {results_train[0]:.3} \nTraining Accuracy: {results_train[1]:.3}')

1051/1051 [==============================] - 0s 31us/step
----------
Training Loss: 0.00793 
Training Accuracy: 0.00793


In [71]:
results_train

[0.007933456471081748, 0.00793345645070076]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [72]:
# Evaluate the model on validate data
results_train = normalized_model.evaluate(X_val, y_val_scaled)
print('----------')
print(f'Training Loss: {results_train[0]:.3} \nTraining Accuracy: {results_train[1]:.3}')

263/263 [==============================] - 0s 42us/step
----------
Training Loss: 0.136 
Training Accuracy: 0.136


Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [73]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)
# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

28996.954776608956

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [74]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', 
                          input_shape=(n_features,),
                          kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 124us/step - loss: 0.4587 - mse: 0.4587 - val_loss: 0.2181 - val_mse: 0.2181
Epoch 2/150
1051/1051 [==============================] - 0s 61us/step - loss: 0.2715 - mse: 0.2715 - val_loss: 0.1740 - val_mse: 0.1740
Epoch 3/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.2198 - mse: 0.2198 - val_loss: 0.1641 - val_mse: 0.1641
Epoch 4/150
1051/1051 [==============================] - 0s 54us/step - loss: 0.1908 - mse: 0.1908 - val_loss: 0.1383 - val_mse: 0.1383
Epoch 5/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.1715 - mse: 0.1715 - val_loss: 0.1315 - val_mse: 0.1315
Epoch 6/150
1051/1051 [==============================] - 0s 55us/step - loss: 0.1515 - mse: 0.1515 - val_loss: 0.1284 - val_mse: 0.1284
Epoch 7/150
1051/1051 [==============================] - 0s 55us/step - loss: 0.1434 - mse: 0.1434 - val_loss: 0.1264 - val_mse: 0.1264


1051/1051 [==============================] - 0s 85us/step - loss: 0.0273 - mse: 0.0273 - val_loss: 0.1493 - val_mse: 0.1493
Epoch 61/150
1051/1051 [==============================] - 0s 88us/step - loss: 0.0272 - mse: 0.0272 - val_loss: 0.1495 - val_mse: 0.1495
Epoch 62/150
1051/1051 [==============================] - 0s 85us/step - loss: 0.0265 - mse: 0.0265 - val_loss: 0.1519 - val_mse: 0.1519
Epoch 63/150
1051/1051 [==============================] - 0s 80us/step - loss: 0.0258 - mse: 0.0258 - val_loss: 0.1495 - val_mse: 0.1495
Epoch 64/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0258 - mse: 0.0258 - val_loss: 0.1514 - val_mse: 0.1514
Epoch 65/150
1051/1051 [==============================] - 0s 55us/step - loss: 0.0251 - mse: 0.0251 - val_loss: 0.1487 - val_mse: 0.1487
Epoch 66/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.0247 - mse: 0.0247 - val_loss: 0.1541 - val_mse: 0.1541
Epoch 67/150
1051/1051 [==============================

Epoch 120/150
1051/1051 [==============================] - 0s 84us/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.1602 - val_mse: 0.1602
Epoch 121/150
1051/1051 [==============================] - 0s 63us/step - loss: 0.0120 - mse: 0.0120 - val_loss: 0.1598 - val_mse: 0.1598
Epoch 122/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0120 - mse: 0.0120 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 123/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1606 - val_mse: 0.1606
Epoch 124/150
1051/1051 [==============================] - 0s 69us/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1595 - val_mse: 0.1595
Epoch 125/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1656 - val_mse: 0.1656
Epoch 126/150
1051/1051 [==============================] - 0s 62us/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1620 - val_mse: 0.1620
Epoch 127/150
1051/1051 [=========

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [75]:
# Evaluate the model on training data
he_results_train = he_model.evaluate(X_train, y_train_scaled)
print(f'Training he_model{he_results_train[0]:.3} \nTraining Accuracy: {he_results_train[1]:.3}')

1051/1051 [==============================] - 0s 35us/step
Training he_model0.00851 
Training Accuracy: 0.00851


Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [24]:
X_test.shape


(263, 296)

In [25]:
X_test_scaled.shape

(263, 36)

In [26]:
y_test.shape

(146, 1)

In [30]:
print( X_train.shape, X_train_scaled.shape, y_train.shape, y_train_scaled.shape)

(1051, 296) (1051, 36) (1051, 1) (1051, 1)


In [31]:
print(X_test.shape, X_test_scaled.shape, y_test.shape, y_test_scaled.shape)

(263, 296) (263, 36) (146, 1) (146, 1)


In [77]:
# Evaluate the model on validate data
he_results_val = he_model.evaluate(X_val, y_val_scaled)
print(f'Val  he_model:  {he_results_val[0]:.3} \n he_model Accuracy: {he_results_val[1]:.3}')

263/263 [==============================] - 0s 45us/step
Val  he_model:  0.164 
 he_model Accuracy: 0.164


## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [78]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation='relu', 
                            input_shape=(n_features,),
                            kernel_initializer='lecun_normal'))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 146us/step - loss: 0.4800 - mse: 0.4800 - val_loss: 0.1752 - val_mse: 0.1752
Epoch 2/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.2367 - mse: 0.2367 - val_loss: 0.1483 - val_mse: 0.1483
Epoch 3/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.1918 - mse: 0.1918 - val_loss: 0.1530 - val_mse: 0.1530
Epoch 4/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.1706 - mse: 0.1706 - val_loss: 0.1199 - val_mse: 0.1199
Epoch 5/150
1051/1051 [==============================] - 0s 59us/step - loss: 0.1521 - mse: 0.1521 - val_loss: 0.1160 - val_mse: 0.1160
Epoch 6/150
1051/1051 [==============================] - 0s 63us/step - loss: 0.1325 - mse: 0.1325 - val_loss: 0.1137 - val_mse: 0.1137
Epoch 7/150
1051/1051 [==============================] - 0s 61us/step - loss: 0.1235 - mse: 0.1235 - val_loss: 0.1126 - val_mse: 0.1126


1051/1051 [==============================] - 0s 69us/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.1031 - val_mse: 0.1031
Epoch 61/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.1029 - val_mse: 0.1029
Epoch 62/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1077 - val_mse: 0.1077
Epoch 63/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.0220 - mse: 0.0220 - val_loss: 0.1031 - val_mse: 0.1031
Epoch 64/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.0218 - mse: 0.0218 - val_loss: 0.1042 - val_mse: 0.1042
Epoch 65/150
1051/1051 [==============================] - 0s 61us/step - loss: 0.0213 - mse: 0.0213 - val_loss: 0.1031 - val_mse: 0.1031
Epoch 66/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.0208 - mse: 0.0208 - val_loss: 0.1051 - val_mse: 0.1051
Epoch 67/150
1051/1051 [==============================

Epoch 120/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1047 - val_mse: 0.1047
Epoch 121/150
1051/1051 [==============================] - 0s 75us/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1038 - val_mse: 0.1038
Epoch 122/150
1051/1051 [==============================] - 0s 83us/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1041 - val_mse: 0.1041
Epoch 123/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1042 - val_mse: 0.1042
Epoch 124/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1025 - val_mse: 0.1025
Epoch 125/150
1051/1051 [==============================] - 0s 90us/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1049 - val_mse: 0.1049
Epoch 126/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.1048 - val_mse: 0.1048
Epoch 127/150
1051/1051 [=========

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [79]:
# Evaluate the model on training data
lecun_train_results = lecun_model.evaluate(X_train, y_train_scaled)
print(lecun_train_results)

1051/1051 [==============================] - 0s 28us/step
[0.007282943081564414, 0.007282942999154329]


Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [80]:
# Evaluate the model on validate data
lecun_val_results = lecun_model.evaluate(X_val, y_val_scaled)
print(lecun_val_results)

263/263 [==============================] - 0s 46us/step
[0.1062863032854102, 0.10628630220890045]


Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [81]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop',
               loss='mse',
               metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 132us/step - loss: 0.3527 - mse: 0.3527 - val_loss: 0.1498 - val_mse: 0.1498
Epoch 2/150
1051/1051 [==============================] - 0s 67us/step - loss: 0.1624 - mse: 0.1624 - val_loss: 0.1215 - val_mse: 0.1215
Epoch 3/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.1192 - mse: 0.1192 - val_loss: 0.1291 - val_mse: 0.1291
Epoch 4/150
1051/1051 [==============================] - 0s 64us/step - loss: 0.0979 - mse: 0.0979 - val_loss: 0.1129 - val_mse: 0.1129
Epoch 5/150
1051/1051 [==============================] - 0s 63us/step - loss: 0.0820 - mse: 0.0820 - val_loss: 0.1235 - val_mse: 0.1235
Epoch 6/150
1051/1051 [==============================] - 0s 60us/step - loss: 0.0660 - mse: 0.0660 - val_loss: 0.1337 - val_mse: 0.1337
Epoch 7/150
1051/1051 [==============================] - 0s 63us/step - loss: 0.0557 - mse: 0.0557 - val_loss: 0.1409 - val_mse: 0.1409


1051/1051 [==============================] - 0s 76us/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.1259 - val_mse: 0.1259
Epoch 61/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0143 - mse: 0.0143 - val_loss: 0.1753 - val_mse: 0.1753
Epoch 62/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0077 - mse: 0.0077 - val_loss: 0.1160 - val_mse: 0.1160
Epoch 63/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1568 - val_mse: 0.1568
Epoch 64/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.0138 - mse: 0.0138 - val_loss: 0.1139 - val_mse: 0.1139
Epoch 65/150
1051/1051 [==============================] - 0s 87us/step - loss: 0.0070 - mse: 0.0070 - val_loss: 0.1323 - val_mse: 0.1323
Epoch 66/150
1051/1051 [==============================] - 0s 92us/step - loss: 0.0085 - mse: 0.0085 - val_loss: 0.1297 - val_mse: 0.1297
Epoch 67/150
1051/1051 [==============================

Epoch 120/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.0088 - mse: 0.0088 - val_loss: 0.1081 - val_mse: 0.1081
Epoch 121/150
1051/1051 [==============================] - 0s 59us/step - loss: 0.0062 - mse: 0.0062 - val_loss: 0.1018 - val_mse: 0.1018
Epoch 122/150
1051/1051 [==============================] - 0s 92us/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1056 - val_mse: 0.1056
Epoch 123/150
1051/1051 [==============================] - 0s 108us/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.0944 - val_mse: 0.0944
Epoch 124/150
1051/1051 [==============================] - 0s 109us/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.1051 - val_mse: 0.1051
Epoch 125/150
1051/1051 [==============================] - 0s 112us/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.1102 - val_mse: 0.1102
Epoch 126/150
1051/1051 [==============================] - 0s 116us/step - loss: 0.0060 - mse: 0.0060 - val_loss: 0.1068 - val_mse: 0.1068
Epoch 127/150
1051/1051 [=====

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [82]:
# Evaluate the model on training data
rmsprop_train_results = rmsprop_model.evaluate(X_train, y_train_scaled)
print(rmsprop_train_results)

1051/1051 [==============================] - 0s 32us/step
[0.006159958034642147, 0.006159957963973284]


Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [83]:
# Evaluate the model on validate data
rmsprop_val_results = rmsprop_model.evaluate(X_val, y_val_scaled)
print(rmsprop_val_results)

263/263 [==============================] - 0s 50us/step
[0.10581592334427307, 0.10581591725349426]


## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [84]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam',
                  loss='mse',
                  metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 201us/step - loss: 0.3842 - mse: 0.3842 - val_loss: 0.1675 - val_mse: 0.1675
Epoch 2/150
1051/1051 [==============================] - 0s 88us/step - loss: 0.1606 - mse: 0.1606 - val_loss: 0.1226 - val_mse: 0.1226
Epoch 3/150
1051/1051 [==============================] - 0s 79us/step - loss: 0.1098 - mse: 0.1098 - val_loss: 0.1093 - val_mse: 0.1093
Epoch 4/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0806 - mse: 0.0806 - val_loss: 0.1227 - val_mse: 0.1227
Epoch 5/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0707 - mse: 0.0707 - val_loss: 0.1051 - val_mse: 0.1051
Epoch 6/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.0564 - mse: 0.0564 - val_loss: 0.1044 - val_mse: 0.1044
Epoch 7/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0463 - mse: 0.0463 - val_loss: 0.1101 - val_mse: 0.1101


Epoch 60/150
1051/1051 [==============================] - 0s 81us/step - loss: 6.1934e-04 - mse: 6.1934e-04 - val_loss: 0.1201 - val_mse: 0.1201
Epoch 61/150
1051/1051 [==============================] - 0s 78us/step - loss: 4.7357e-04 - mse: 4.7357e-04 - val_loss: 0.1211 - val_mse: 0.1211
Epoch 62/150
1051/1051 [==============================] - 0s 75us/step - loss: 4.9802e-04 - mse: 4.9801e-04 - val_loss: 0.1212 - val_mse: 0.1212
Epoch 63/150
1051/1051 [==============================] - 0s 79us/step - loss: 4.8499e-04 - mse: 4.8499e-04 - val_loss: 0.1200 - val_mse: 0.1200
Epoch 64/150
1051/1051 [==============================] - 0s 87us/step - loss: 5.8194e-04 - mse: 5.8194e-04 - val_loss: 0.1215 - val_mse: 0.1215
Epoch 65/150
1051/1051 [==============================] - 0s 89us/step - loss: 6.4062e-04 - mse: 6.4062e-04 - val_loss: 0.1196 - val_mse: 0.1196
Epoch 66/150
1051/1051 [==============================] - 0s 92us/step - loss: 7.9021e-04 - mse: 7.9021e-04 - val_loss: 0.1211 - v

1051/1051 [==============================] - 0s 66us/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.1109 - val_mse: 0.1109
Epoch 119/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.1149 - val_mse: 0.1149
Epoch 120/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.1132 - val_mse: 0.1132
Epoch 121/150
1051/1051 [==============================] - ETA: 0s - loss: 0.0018 - mse: 0.001 - 0s 67us/step - loss: 0.0017 - mse: 0.0017 - val_loss: 0.1176 - val_mse: 0.1176
Epoch 122/150
1051/1051 [==============================] - 0s 79us/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.1088 - val_mse: 0.1088
Epoch 123/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.0020 - mse: 0.0020 - val_loss: 0.1182 - val_mse: 0.1182
Epoch 124/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.0024 - mse: 0.0024 - val_loss: 0.1029 - val_mse: 0.1029
Epoch 125/

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [85]:
# Evaluate the model on training data
adam_train_res = adam_model.evaluate(X_train, y_train_scaled)
print(adam_train_res)

1051/1051 [==============================] - 0s 35us/step
[0.00010923309452485268, 0.00010923308582277969]


Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [86]:
# Evaluate the model on validate data
adam_val_res = adam_model.evaluate(X_val, y_val_scaled)
print(adam_val_res)

263/263 [==============================] - 0s 40us/step
[0.11196559916884273, 0.1119655892252922]


## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [87]:
# Evaluate the best model on test datahe_results_train
print(f'he: {he_results_train} , {he_results_val}')
print('==================')
print(f'rms: {rmsprop_train_results} , {rmsprop_val_results}')
print('==================')
print(f'lecun: {lecun_train_results} , {lecun_val_results}')
print('==================')
print(f'adam: {adam_train_res} , {adam_val_res}')

he: [0.008513452826363772, 0.008513453416526318] , [0.16375007358567342, 0.1637500673532486]
rms: [0.006159958034642147, 0.006159957963973284] , [0.10581592334427307, 0.10581591725349426]
lecun: [0.007282943081564414, 0.007282942999154329] , [0.1062863032854102, 0.10628630220890045]
adam: [0.00010923309452485268, 0.00010923308582277969] , [0.11196559916884273, 0.1119655892252922]


In [ ]:
print(X_train.shape, X_train_scaled.shape, y_train.shape, y_train_scaled.shape)

In [ ]:
X_test.shape

In [ ]:
y_test_scaled.shape

In [89]:
adam_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 62us/step


[0.15023449675677575, 0.15023449063301086]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [88]:
# Generate predictions on test data
y_test_pred_scaled = adam_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
MSE = np.sqrt(mean_squared_error(y_test,y_test_pred))
print(MSE)

30459.04885764731


## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.